![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [946]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [947]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [948]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [949]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [950]:
df_train.info()

In [951]:
df_train.head(5)

In [952]:
df_test.info()

In [953]:
df_test.head(5)

In [954]:
sample_submission.head(5)

In [955]:
sample_submission.info()

In [956]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [957]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [958]:
data.sample(5)

In [959]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [960]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [961]:
data['Number_of_Reviews_isNAN']

In [962]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [963]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [964]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [965]:
data.head(5)

In [966]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [967]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [968]:
# Обработка 'Price Range'
def normalise_price_range(str):
    dict = {"$": 1, "$$ - $$$": 2, "$$$$": 3}
    return dict[str] if str in dict.keys() else 0

data['Price Range'] = data['Price Range'].apply(lambda x : normalise_price_range(x))


In [969]:
data['Price Range'].value_counts()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [970]:
# обработка колонки Reviews
import re
from datetime import datetime, timedelta

def parse_review_dates(str):
    pattern = "(\d{2}/\d{2}/\d{4})', '(\d{2}/\d{2}/\d{4})"
    match = re.search(pattern, str) 

    if match:
        dates = match.group().replace("', '", ",").split(',')
        if len(dates) != 2:
            return [0, 0]
        return [datetime.strptime(dates[0], '%m/%d/%Y').timestamp(), datetime.strptime(dates[1], '%m/%d/%Y').timestamp()]
    else:
        return [0, 0]
    
data['Reviews'].fillna("", inplace=True)
data['Review1_Epoch'] = data['Reviews'].apply(lambda x : parse_review_dates(x)[0])
data['Review2_Epoch'] = data['Reviews'].apply(lambda x : parse_review_dates(x)[1])
data.head()

In [971]:
# добавление колонки Between_Reviews (разница в секундах между ревью)
def calc_between_reviews(str):
    dates = parse_review_dates(str)
    if dates[0] == 0 or dates[1] == 0:
        return 0
    else:
        return abs(dates[0] - dates[1])

data['Between_Reviews'] = data['Reviews'].apply(lambda x : calc_between_reviews(x))

In [972]:
# обработаем 'Cuisine Style'
def parse_cuisines(str):
    str = str.replace("['", "").replace("']", "").replace("', '", ",")
    return str.split(",")

cuisines = set()

for value in data['Cuisine Style'].dropna().unique():
    for cuisine in parse_cuisines(value):
        cuisines.add(cuisine)

print(len(cuisines))


In [973]:
data['Cuisine Style'].fillna('', inplace=True)
data['Cuisines'] = data['Cuisine Style'].apply(lambda x : parse_cuisines(x))

def find_item(cell):
    if cuisine in cell:
        return 1
    return 0

for cuisine in cuisines:
    data[cuisine] = data['Cuisines'].apply(find_item)


In [974]:
data.sample(5)

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [975]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [976]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [977]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [978]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


In [979]:
# количество ресторанов в городе
df_train['RestaurantsInCity'] = df_train.groupby('City')['Restaurant_id'].transform('count')
df_train.sample(5)

In [980]:
# нормализуем Ranking на количество ресторанов в городе
df_train['NormalisedRanking'] = df_train['Ranking'] / df_train['RestaurantsInCity']
df_train.sample(5)

In [981]:
# проверим нормализацию Ranking
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['NormalisedRanking'][df_train['City'] == x].hist(bins=100)
plt.show()

In [982]:
# нормализуем 'Number of Reviews' на количество ресторанов в городе
df_train['NormalisedNumberOfReviews'] = df_train['Number of Reviews'] / df_train['RestaurantsInCity']
df_train.sample(5)

In [983]:
# проверим нормализацию 'Number of Reviews'
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['NormalisedNumberOfReviews'][df_train['City'] == x].hist(bins=4)
plt.show()

### Посмотрим распределение целевой переменной

In [984]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [985]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [986]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [987]:
plt.rcParams['figure.figsize'] = (25,20)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [988]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [989]:
import re
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures 

def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    
    # опечатка
    df_output.City = df_output.City.replace(to_replace={'Oporto': 'Porto'})
    
    # добавление признака RestaurantsInCity (количество ресторанов в городе)
    df_output['RestaurantsInCity'] = df_output.groupby('City')['Restaurant_id'].transform('count')
    
    # добавление признака PriceRangeNan (указан Price Range?) - УХУДШИЛО!
    # df_output['PriceRangeNan'] = pd.isna(df_output['Price Range']).astype('uint8')    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Cuisine Style'].fillna('', inplace=True)
    
    
    # ################### 3. Encoding ############################################################## 
    
    # колонка 'City'
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
        
    # колонка 'Price Range'
    def normalise_price_range(str):
        dict = {"$": 1, "$$ - $$$": 2, "$$$$": 3}
        return dict[str] if str in dict.keys() else 0

    df_output['Price Range'] = df_output['Price Range'].apply(lambda x : normalise_price_range(x))    
    
    # обработаем 'Cuisine Style'
    def parse_cuisines(str):
        str = str.replace("['", "").replace("']", "").replace("', '", ",")
        return str.split(",")

    cuisines = set()

    for value in data['Cuisine Style'].dropna().unique():
        for cuisine in parse_cuisines(value):
            cuisines.add(cuisine)
        
    df_output['Cuisines'] = df_output['Cuisine Style'].apply(lambda x : parse_cuisines(x))

    def find_item(cell):
        if cuisine in cell:
            return 1
        return 0

    for cuisine in cuisines:
        df_output[cuisine] = df_output['Cuisines'].apply(find_item)
    
    # ################### 4. Feature Engineering ####################################################

    # добавление признака Review1_Epoch и Review2_Epoch (секунд между первым и вторым ревью и Unix epoch time)
    def parse_review_dates(str):
        pattern = "(\d{2}/\d{2}/\d{4})', '(\d{2}/\d{2}/\d{4})"
        match = re.search(pattern, str) 

        if match:
            dates = match.group().replace("', '", ",").split(',')
            if len(dates) != 2:
                return [0, 0]
            return [datetime.strptime(dates[0], '%m/%d/%Y').timestamp(), datetime.strptime(dates[1], '%m/%d/%Y').timestamp()]
        else:
            return [0, 0]
    
    df_output['Reviews'].fillna("", inplace=True)
    df_output['Review1_Epoch'] = df_output['Reviews'].apply(lambda x : parse_review_dates(x)[0])
    df_output['Review2_Epoch'] = df_output['Reviews'].apply(lambda x : parse_review_dates(x)[1])    
    
    # добавление признака Between_Reviews (разница в секундах между ревью)
    def calc_between_reviews(str):
        dates = parse_review_dates(str)
        if dates[0] == 0 or dates[1] == 0:
            return 0
        else:
            return abs(dates[0] - dates[1])

    df_output['Between_Reviews'] = df_output['Reviews'].apply(lambda x : calc_between_reviews(x))
    
    # добавление признака NormalisedRanking (нормализуем Ranking на количество ресторанов в городе)
    df_output['NormalisedRanking'] = df_output['Ranking'] / df_output['RestaurantsInCity']
    
    # добавление признака NormalisedNumberOfReviews (нормализуем 'Number of Reviews' на количество ресторанов в городе) - УХУДШИЛО!
    # df_output['NormalisedNumberOfReviews'] = df_output['Number of Reviews'] / df_output['RestaurantsInCity']
    
    # ################### 5. Clean #################################################### 
    
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id', 'ID_TA'], axis = 1, inplace=True)
    
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [990]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [991]:
df_preproc.info()

In [992]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [993]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [994]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [995]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [996]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [997]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [998]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [999]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [1000]:
test_data.sample(10)

In [1001]:
test_data = test_data.drop(['Rating'], axis=1)

In [1002]:
sample_submission

In [1003]:
predict_submission = model.predict(test_data)

In [1004]:
predict_submission

In [1005]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
